In [ ]:
import os
import glob
from dotenv import load_dotenv
import gradio as gr
from google import genai

In [ ]:
load_dotenv(override=True)

In [ ]:
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
gemini = genai.Client(api_key=GEMINI_API_KEY)

In [ ]:
context = {}

In [ ]:
employees = glob.glob("./notebooks/knowledge-base/employees/*")

for employee in employees:
    name = employee.split(" ")[-1][:-3]
    doc = ""
    with open(employee, "r", encoding="utf-8") as f:
        doc = f.read()
    context[name] = doc

In [ ]:
products = glob.glob("./notebooks/knowledge-base/products/*")

for product in products:
    product_name = product.split("/")[-1][:-3]
    doc = ""
    with open(product, "r", encoding="utf-8") as f:
        doc = f.read()
    context[product_name] = doc

In [ ]:
system_message = "You are an expert in answering accurate questions about Insurellm, the Insurance Tech company. Give brief, accurate answers. If you don't know the answer, say so. Do not make anything up if you haven't been provided with relevant context."

In [ ]:
def get_relevant_context(message):
    relevant_context = []
    for context_title, context_details in context.items():
        if context_title.lower() in message.lower():
            relevant_context.append(context_details)
    return relevant_context

In [ ]:
def add_context(message):
    relevant_context = get_relevant_context(message)

    result_message = message

    if relevant_context:
        result_message += "\n\nThe following additional context might be relevant in answering this question:\n\n"
        for relevant in relevant_context:
            result_message += relevant + "\n\n"
    return result_message

In [ ]:
def getChatItem(role, prompt):
    return {"role": role, "parts": [{"text": prompt}]}


def chat(message, history):
    contents = []
    for item in history:
        contents.append(getChatItem(item["role"], item["content"]))
    contents.append(getChatItem("user", add_context(message)))

    response = gemini.models.generate_content_stream(
        model="gemini-2.0-flash",
        contents=contents,
        config={"system_instruction": {"text": system_message}},
    )

    result = ""
    for chunk in response:
        result += chunk.candidates[0].content.parts[0].text or ""
        yield result

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()